In [2]:
import pandas as pd
import glob
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle

def load_and_preprocess_data(csv_directory):
    # Use glob to get a list of all CSV files in the directory
    csv_files = glob.glob(csv_directory)

    # Initialize an empty list to store DataFrames
    dfs = []

    # Iterate through the CSV files and read them into DataFrames
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        dfs.append(df)

    # Concatenate all DataFrames into one
    combined_df = pd.concat(dfs, ignore_index=True)

    # Remove specified columns
    combined_df.drop(combined_df.columns[1:2], axis=1, inplace=True)

    # Shuffle the rows
    shuffled_df = combined_df.sample(frac=1, random_state=42)

    return shuffled_df

def split_data(shuffled_df):
    # Define the features and target variable
    X = shuffled_df.drop('Fatigue or not',axis=1)
    y = shuffled_df['Fatigue or not']

    # Split the data into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test

def train_and_evaluate_model(X_train, X_val, X_test, y_train, y_val, y_test):
    # Create an XGBoost classifier
    model = xgb.XGBClassifier()

    # Train the model on the training data
    model.fit(X_train, y_train)

    # Evaluate on validation set
    y_val_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_val_pred)
    report = classification_report(y_val, y_val_pred)
    print(f"Validation Set Accuracy: {accuracy}")
    print("Classification Report:\n", report)

    # Evaluate on test set
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_report = classification_report(y_test, y_test_pred)
    print(f"Test Set Accuracy: {test_accuracy}")
    print("Test Set Classification Report:\n", test_report)

    return model

def save_model(model, model_file_path):
    # Save the trained model to the file
    with open(model_file_path, "wb") as model_file:
        pickle.dump(model, model_file)

def load_model(model_file_path):
    # Load the saved model from the file
    with open(model_file_path, "rb") as model_file:
        loaded_model = pickle.load(model_file)
    return loaded_model

def main():
    csv_directory = 'training_data/*.csv'
    shuffled_df = load_and_preprocess_data(csv_directory)
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(shuffled_df)
    model = train_and_evaluate_model(X_train, X_val, X_test, y_train, y_val, y_test)
    model_file_path = "xgboost_model.pkl"
    save_model(model, model_file_path)
    loaded_model = load_model(model_file_path)

    # Assuming new_row is a DataFrame containing your new data
    # new_data_predictions = loaded_model.predict(new_row)
    # print(new_data_predictions)

if __name__ == "__main__":
    main()


Validation Set Accuracy: 0.9911111111111112
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       581
           1       0.99      0.99      0.99       994

    accuracy                           0.99      1575
   macro avg       0.99      0.99      0.99      1575
weighted avg       0.99      0.99      0.99      1575

Test Set Accuracy: 0.9923857868020305
Test Set Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       572
           1       0.99      1.00      0.99      1004

    accuracy                           0.99      1576
   macro avg       0.99      0.99      0.99      1576
weighted avg       0.99      0.99      0.99      1576



In [29]:
import pandas as pd
import pickle
import xgboost as xgb

def load_model(model_file_path):
    # Load the saved model from the file
    with open(model_file_path, "rb") as model_file:
        loaded_model = pickle.load(model_file)
    return loaded_model

def get_single_row_data(data_file_path, row_index):
    # Load the data from the file
    data = pd.read_csv(data_file_path)
    
    # Get a single row of data
    single_row = data.iloc[row_index:row_index+1]
    
    print(single_row)
    # Drop the column
    single_row = single_row.drop(columns=['Fatigue or not'], errors='ignore')
    print(single_row)
    
    return single_row

def make_prediction(loaded_model, single_row):
    # Make a prediction on the single row of data
    prediction = loaded_model.predict(single_row)
    
    return prediction

def main():
    model_file_path = 'xgboost_model.pkl'
    data_file_path = 'training_data/frontal_fatigue_pose_sideway.csv'
    row_index = 120  # Index of the row
    
    loaded_model = load_model(model_file_path)
    single_row = get_single_row_data(data_file_path, row_index)
    prediction = make_prediction(loaded_model, single_row)
    
    print(f'The prediction for row {row_index} is: {prediction[0]}')

if __name__ == '__main__':
    main()

     Employee ID  Frame   Box x  Box y  Box width  Box height  \
120            1     61  1007.0  588.0      184.0       206.0   

     Width-to-height ratio  \
120               0.893204   

     right_shoulder&right_elbow - RIGHT_ELBOW - right_elbow&right_wrist  \
120                                              168.1                    

     right_shoulder&right_elbow - RIGHT_SHOULDER - right_shoulder&left_shoulder  \
120                                               31.8                            

     right_shoulder&right_elbow - RIGHT_SHOULDER - right_shoulder&right_hip  \
120                                               74.1                        

     ...  right_hip&right_knee - RIGHT_HIP - right_hip&left_hip  \
120  ...                                                4.0       

     right_hip&right_knee - RIGHT_HIP - right_shoulder&right_hip  \
120                                               54.8             

     left_hip&left_knee - LEFT_KNEE - left_knee&left_ankle 

In [1]:
# Get the output data, save it as a table
import os
import pandas as pd
import matplotlib.pyplot as plt
from pydantic import BaseModel
import numpy as np
from ultralytics import YOLO

model = YOLO('yolov8n-pose.pt')
footage = 'videos/test.mp4'
# results = model(footage)
results = model.predict(footage, save=False, conf=0.5)

output_name = 'training_data/1.csv'
fatigue_or_not = '-1'

model_file_path = 'xgboost_model.pkl'

def load_model(model_file_path):
    # Load the saved model from the file
    with open(model_file_path, "rb") as model_file:
        loaded_model = pickle.load(model_file)
    return loaded_model


class GetKeypoint(BaseModel):
    NOSE:           int = 0
    LEFT_EYE:       int = 1
    RIGHT_EYE:      int = 2
    LEFT_EAR:       int = 3
    RIGHT_EAR:      int = 4
    LEFT_SHOULDER:  int = 5
    RIGHT_SHOULDER: int = 6
    LEFT_ELBOW:     int = 7
    RIGHT_ELBOW:    int = 8
    LEFT_WRIST:     int = 9
    RIGHT_WRIST:    int = 10
    LEFT_HIP:       int = 11
    RIGHT_HIP:      int = 12
    LEFT_KNEE:      int = 13
    RIGHT_KNEE:     int = 14
    LEFT_ANKLE:     int = 15
    RIGHT_ANKLE:    int = 16
        
class BodyKeypoints:
    def __init__(self, keypoints):
        get_keypoint = GetKeypoint()
        self.NOSE = keypoints[get_keypoint.NOSE]
        self.LEFT_EYE = keypoints[get_keypoint.LEFT_EYE]
        self.RIGHT_EYE = keypoints[get_keypoint.RIGHT_EYE]
        self.LEFT_EAR = keypoints[get_keypoint.LEFT_EAR]
        self.RIGHT_EAR = keypoints[get_keypoint.RIGHT_EAR]
        self.LEFT_SHOULDER = keypoints[get_keypoint.LEFT_SHOULDER]
        self.RIGHT_SHOULDER = keypoints[get_keypoint.RIGHT_SHOULDER]
        self.LEFT_ELBOW = keypoints[get_keypoint.LEFT_ELBOW]
        self.RIGHT_ELBOW = keypoints[get_keypoint.RIGHT_ELBOW]
        self.LEFT_WRIST = keypoints[get_keypoint.LEFT_WRIST]
        self.RIGHT_WRIST = keypoints[get_keypoint.RIGHT_WRIST]
        self.LEFT_HIP = keypoints[get_keypoint.LEFT_HIP]
        self.RIGHT_HIP = keypoints[get_keypoint.RIGHT_HIP]
        self.LEFT_KNEE = keypoints[get_keypoint.LEFT_KNEE]
        self.RIGHT_KNEE = keypoints[get_keypoint.RIGHT_KNEE]
        self.LEFT_ANKLE = keypoints[get_keypoint.LEFT_ANKLE]
        self.RIGHT_ANKLE = keypoints[get_keypoint.RIGHT_ANKLE]
        
def get_limbs_from_keypoints(keypoints: BodyKeypoints):
    limbs = {
        "right_eye&nose": [keypoints.RIGHT_EYE, keypoints.NOSE],
        "right_eye&right_ear": [keypoints.RIGHT_EYE, keypoints.RIGHT_EAR],
        "left_eye&nose": [keypoints.LEFT_EYE, keypoints.NOSE],
        "left_eye&left_ear": [keypoints.LEFT_EYE, keypoints.LEFT_EAR],
        "right_shoulder&right_elbow": [keypoints.RIGHT_SHOULDER, keypoints.RIGHT_ELBOW],
        "right_elbow&right_wrist": [keypoints.RIGHT_ELBOW, keypoints.RIGHT_WRIST],
        "left_shoulder&left_elbow": [keypoints.LEFT_SHOULDER, keypoints.LEFT_ELBOW],
        "left_elbow&left_wrist": [keypoints.LEFT_ELBOW, keypoints.LEFT_WRIST],
        "right_hip&right_knee": [keypoints.RIGHT_HIP, keypoints.RIGHT_KNEE],
        "right_knee&right_ankle": [keypoints.RIGHT_KNEE, keypoints.RIGHT_ANKLE],
        "left_hip&left_knee": [keypoints.LEFT_HIP, keypoints.LEFT_KNEE],
        "left_knee&left_ankle": [keypoints.LEFT_KNEE, keypoints.LEFT_ANKLE],
        "right_shoulder&left_shoulder": [keypoints.RIGHT_SHOULDER, keypoints.LEFT_SHOULDER],
        "right_hip&left_hip": [keypoints.RIGHT_HIP, keypoints.LEFT_HIP],
        "right_shoulder&right_hip": [keypoints.RIGHT_SHOULDER, keypoints.RIGHT_HIP],
        "left_shoulder&left_hip": [keypoints.LEFT_SHOULDER, keypoints.LEFT_HIP]
    }
    return limbs

body_keypoints_example = BodyKeypoints([[0,0]] * 17)
limbs = get_limbs_from_keypoints(body_keypoints_example)
def calculate_and_get_angles_as_dict(keypoints_coords, limbs):
    angle_information = {}
    
    for limb_1_name, limb_2_name in adjacent_limbs:
        limb_1 = limbs[limb_1_name]
        limb_2 = limbs[limb_2_name]
        
        common_point = None
        for point in limb_1:
            if any(np.array_equal(point, p) for p in limb_2):
                common_point = point
                break
        
        if common_point is None:
            continue
        
        vector_A = np.array(limb_1[1]) - np.array(limb_1[0]) if np.array_equal(limb_1[1], common_point) else np.array(limb_1[0]) - np.array(limb_1[1])
        vector_B = np.array(limb_2[1]) - np.array(limb_2[0]) if np.array_equal(limb_2[1], common_point) else np.array(limb_2[0]) - np.array(limb_2[1])
        
        angle = calculate_angle(vector_A, vector_B)
        key = f"{limb_1_name} - {KEYPOINT_LABELS[np.where((result_keypoint == common_point).all(axis=1))[0][0]]} - {limb_2_name}"
        value = f"{angle:.1f}°"
        
        angle_information[key] = value
        
    return angle_information

def find_closest_person_distance(prev_coordinates, current_coordinate):
    """
    Returns the index of the person from the previous frame whose coordinates 
    are closest to the current_coordinate and the corresponding distance.
    """
    distances = [(idx, np.linalg.norm(np.array(prev_coord) - np.array(current_coordinate)))
                 for idx, prev_coord in enumerate(prev_coordinates) if prev_coord is not None]
    if distances:
        min_distance_item = min(distances, key=lambda x: x[1])
        return min_distance_item
    return None, float('inf')

adjacent_limbs = []

selected_limbs = [
        "right_shoulder&right_elbow",
        "right_elbow&right_wrist",
        "left_shoulder&left_elbow",
        "left_elbow&left_wrist",
        "right_hip&right_knee",
        "right_knee&right_ankle",
        "left_hip&left_knee",
        "left_knee&left_ankle",
        "right_shoulder&left_shoulder",
        "right_hip&left_hip",
        "right_shoulder&right_hip",
        "left_shoulder&left_hip"
    ]

# Generating all possible combinations
for i in range(len(selected_limbs)):
    for j in range(i + 1, len(selected_limbs)):
        limb1 = limbs[selected_limbs[i]]
        limb2 = limbs[selected_limbs[j]]
        
        # Convert keypoints to tuples for comparison
        limb1_tuples = [tuple(point) for point in limb1]
        limb2_tuples = [tuple(point) for point in limb2]

        # Checking if any keypoint in the second limb matches with the keypoints in the first limb
        if any(kp in limb1_tuples for kp in limb2_tuples):
            adjacent_limbs.append((selected_limbs[i], selected_limbs[j]))

KEYPOINT_LABELS = [
    "NOSE", "LEFT_EYE", "RIGHT_EYE", "LEFT_EAR", "RIGHT_EAR", "LEFT_SHOULDER",
    "RIGHT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW", "LEFT_WRIST", "RIGHT_WRIST",
    "LEFT_HIP", "RIGHT_HIP", "LEFT_KNEE", "RIGHT_KNEE", "LEFT_ANKLE", "RIGHT_ANKLE"
]

angle_keys = [
    'right_shoulder&right_elbow - RIGHT_ELBOW - right_elbow&right_wrist',
    'right_shoulder&right_elbow - RIGHT_SHOULDER - right_shoulder&left_shoulder',
    'right_shoulder&right_elbow - RIGHT_SHOULDER - right_shoulder&right_hip',
    'left_shoulder&left_elbow - LEFT_ELBOW - left_elbow&left_wrist',
    'left_shoulder&left_elbow - LEFT_SHOULDER - right_shoulder&left_shoulder',
    'left_shoulder&left_elbow - LEFT_SHOULDER - left_shoulder&left_hip',
    'right_hip&right_knee - RIGHT_KNEE - right_knee&right_ankle',
    'right_hip&right_knee - RIGHT_HIP - right_hip&left_hip',
    'right_hip&right_knee - RIGHT_HIP - right_shoulder&right_hip',
    'left_hip&left_knee - LEFT_KNEE - left_knee&left_ankle',
    'left_hip&left_knee - LEFT_HIP - right_hip&left_hip',
    'left_hip&left_knee - LEFT_HIP - left_shoulder&left_hip',
    'right_shoulder&left_shoulder - RIGHT_SHOULDER - right_shoulder&right_hip',
    'right_shoulder&left_shoulder - LEFT_SHOULDER - left_shoulder&left_hip',
    'right_hip&left_hip - RIGHT_HIP - right_shoulder&right_hip',
    'right_hip&left_hip - LEFT_HIP - left_shoulder&left_hip'
]

def calculate_angle(A, B):
#     Calculate angle between two vectors A and B.
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return np.arccos(dot_product / (norm_A * norm_B)) * (180/np.pi)



prev_person_coordinates = [None, None]  # Assuming at most two employees, initialize with None

# Create an empty DataFrame with the required columns
columns = ["Employee ID", "Frame", "Box x", "Box y", "Box width", "Box height", "Width-to-height ratio"] + angle_keys + ["Fatigue or not"]

# columns = ["Employee ID", "Frame", "Box x", "Box y", "Box width", "Box height", "Width-to-height ratio"] + [f"{keypoint} (x, y)" for keypoint in KEYPOINT_LABELS] + angle_keys + ["Fatigue or not"]

df = pd.DataFrame(columns=columns)

# Iterate through results to get bounding boxes and keypoints data
for idx, result in enumerate(results):
    keypoints_data = result.keypoints.xy.cpu().numpy() if hasattr(result, 'keypoints') else []
    boxes_data = result.boxes.xywh.cpu().numpy() if hasattr(result, 'boxes') else [] # Extract bounding boxes

    distances_and_indices = []

    # For each person, get the keypoints and determine which "employee" they match based on the nose's position
    for person_idx, result_keypoint in enumerate(keypoints_data):
        if len(result_keypoint) == 0:
            continue
        get_keypoint = GetKeypoint()
        nose_coordinate = tuple(result_keypoint[get_keypoint.NOSE])
        matched_person_idx, distance = find_closest_person_distance(prev_person_coordinates, nose_coordinate)
        distances_and_indices.append((distance, matched_person_idx, person_idx, result_keypoint))

    sorted_matches = sorted(distances_and_indices, key=lambda x: x[0])[:2]

    for distance, matched_person_idx, person_idx, result_keypoint in sorted_matches:
        if matched_person_idx is not None:
            prev_person_coordinates[matched_person_idx] = tuple(result_keypoint[get_keypoint.NOSE])

        body_keypoints = BodyKeypoints(result_keypoint)
        limbs = get_limbs_from_keypoints(body_keypoints)
        angle_info_dict = calculate_and_get_angles_as_dict(result_keypoint, limbs)

        # Construct the row data to be appended to the DataFrame
        row_data = {
            "Employee ID": person_idx + 1,
            "Frame": idx + 1,
        }
#         for label, keypoint in zip(KEYPOINT_LABELS, result_keypoint):
#             row_data[f"{label} (x, y)"] = tuple(keypoint)
        for angle_key in angle_keys:
            angle_value = angle_info_dict.get(angle_key, None)
            if angle_value:
                angle_value = ''.join([char for char in angle_value if char.isdigit() or char == '.'])
            row_data[angle_key] = angle_value

        # Include bounding box data for this person
        box = boxes_data[person_idx] if len(boxes_data) > person_idx else [None, None, None, None]
        # row_data["Bounding Box (x, y, w, h)"] = tuple(box)
        row_data["Box x"] = box[0] # The YOLO model frames the object in a box, and x refers to the x-coordinate of the diagonal intersection (geometric center) of the box.
        row_data["Box y"] = box[1]
        row_data["Box width"] = box[2]
        row_data["Box height"] = box[3]
        row_data["Width-to-height ratio"] = box[2]/box[3] # The width-to-height ratio of the box
        

        row_data['Fatigue or not'] = fatigue_or_not

        # Append the data to the DataFrame
        df = df.append(row_data, ignore_index=True)
        df = df.tail(1)
#         print(df)
        df = df.drop(columns=['Fatigue or not'], errors='ignore')
        df.drop(df.columns[1:6], axis=1, inplace=True)
        df = df.astype(float, errors='ignore')
#         print(df)
        prediction = loaded_model.predict(df)
        print(prediction)

# Print out the table representation
# print(df)
# save DataFrame to a CSV file
df.to_csv(output_name, index=False)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
video 1/1 (1/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 2 persons, 249.5ms
video 1/1 (2/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 2 persons, 69.6ms
video 1/1 (3/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 1 person, 68.1ms
video 1/1 (4/

video 1/1 (63/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 1 person, 108.4ms
video 1/1 (64/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 2 persons, 106.6ms
video 1/1 (65/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 2 persons, 127.1ms
video 1/1 (66/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 2 persons, 107.6ms
video 1/1 (67/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 1 person, 88.1ms
video 1/1 (68/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 2 persons, 103.5ms
video 1/1 (69/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 2 persons, 129.9ms
video 1/1 (70/84) /Users/macbook/Desktop/Codespace/tcs-fatigue-detection/models/videos/test.mp4: 384x640 2 persons, 109.6ms
video 1/1 (

NameError: name 'loaded_model' is not defined

In [3]:
import os
import cv2
import pandas as pd
import numpy as np
import pickle
from ultralytics import YOLO
from pydantic import BaseModel

def load_model(model_file_path):
    # Load the saved model from the file
    with open(model_file_path, "rb") as model_file:
        loaded_model = pickle.load(model_file)
    return loaded_model

# Load models
model = YOLO('yolov8n-pose.pt')
loaded_model = load_model('xgboost_model.pkl')

# Open the test video file
video_capture = cv2.VideoCapture('videos/test_video.mp4')

# Get the dimensions of the video frame
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object to save the annotated video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('annotated_test_video1.mp4', fourcc, 24, (frame_width, frame_height))

fatigue_or_not = '0'

def load_model(model_file_path):
    # Load the saved model from the file
    with open(model_file_path, "rb") as model_file:
        loaded_model = pickle.load(model_file)
    return loaded_model


class GetKeypoint(BaseModel):
    NOSE:           int = 0
    LEFT_EYE:       int = 1
    RIGHT_EYE:      int = 2
    LEFT_EAR:       int = 3
    RIGHT_EAR:      int = 4
    LEFT_SHOULDER:  int = 5
    RIGHT_SHOULDER: int = 6
    LEFT_ELBOW:     int = 7
    RIGHT_ELBOW:    int = 8
    LEFT_WRIST:     int = 9
    RIGHT_WRIST:    int = 10
    LEFT_HIP:       int = 11
    RIGHT_HIP:      int = 12
    LEFT_KNEE:      int = 13
    RIGHT_KNEE:     int = 14
    LEFT_ANKLE:     int = 15
    RIGHT_ANKLE:    int = 16
        
class BodyKeypoints:
    def __init__(self, keypoints):
        get_keypoint = GetKeypoint()
        self.NOSE = keypoints[get_keypoint.NOSE]
        self.LEFT_EYE = keypoints[get_keypoint.LEFT_EYE]
        self.RIGHT_EYE = keypoints[get_keypoint.RIGHT_EYE]
        self.LEFT_EAR = keypoints[get_keypoint.LEFT_EAR]
        self.RIGHT_EAR = keypoints[get_keypoint.RIGHT_EAR]
        self.LEFT_SHOULDER = keypoints[get_keypoint.LEFT_SHOULDER]
        self.RIGHT_SHOULDER = keypoints[get_keypoint.RIGHT_SHOULDER]
        self.LEFT_ELBOW = keypoints[get_keypoint.LEFT_ELBOW]
        self.RIGHT_ELBOW = keypoints[get_keypoint.RIGHT_ELBOW]
        self.LEFT_WRIST = keypoints[get_keypoint.LEFT_WRIST]
        self.RIGHT_WRIST = keypoints[get_keypoint.RIGHT_WRIST]
        self.LEFT_HIP = keypoints[get_keypoint.LEFT_HIP]
        self.RIGHT_HIP = keypoints[get_keypoint.RIGHT_HIP]
        self.LEFT_KNEE = keypoints[get_keypoint.LEFT_KNEE]
        self.RIGHT_KNEE = keypoints[get_keypoint.RIGHT_KNEE]
        self.LEFT_ANKLE = keypoints[get_keypoint.LEFT_ANKLE]
        self.RIGHT_ANKLE = keypoints[get_keypoint.RIGHT_ANKLE]
        
def get_limbs_from_keypoints(keypoints: BodyKeypoints):
    limbs = {
        "right_eye&nose": [keypoints.RIGHT_EYE, keypoints.NOSE],
        "right_eye&right_ear": [keypoints.RIGHT_EYE, keypoints.RIGHT_EAR],
        "left_eye&nose": [keypoints.LEFT_EYE, keypoints.NOSE],
        "left_eye&left_ear": [keypoints.LEFT_EYE, keypoints.LEFT_EAR],
        "right_shoulder&right_elbow": [keypoints.RIGHT_SHOULDER, keypoints.RIGHT_ELBOW],
        "right_elbow&right_wrist": [keypoints.RIGHT_ELBOW, keypoints.RIGHT_WRIST],
        "left_shoulder&left_elbow": [keypoints.LEFT_SHOULDER, keypoints.LEFT_ELBOW],
        "left_elbow&left_wrist": [keypoints.LEFT_ELBOW, keypoints.LEFT_WRIST],
        "right_hip&right_knee": [keypoints.RIGHT_HIP, keypoints.RIGHT_KNEE],
        "right_knee&right_ankle": [keypoints.RIGHT_KNEE, keypoints.RIGHT_ANKLE],
        "left_hip&left_knee": [keypoints.LEFT_HIP, keypoints.LEFT_KNEE],
        "left_knee&left_ankle": [keypoints.LEFT_KNEE, keypoints.LEFT_ANKLE],
        "right_shoulder&left_shoulder": [keypoints.RIGHT_SHOULDER, keypoints.LEFT_SHOULDER],
        "right_hip&left_hip": [keypoints.RIGHT_HIP, keypoints.LEFT_HIP],
        "right_shoulder&right_hip": [keypoints.RIGHT_SHOULDER, keypoints.RIGHT_HIP],
        "left_shoulder&left_hip": [keypoints.LEFT_SHOULDER, keypoints.LEFT_HIP]
    }
    return limbs

body_keypoints_example = BodyKeypoints([[0,0]] * 17)
limbs = get_limbs_from_keypoints(body_keypoints_example)
def calculate_and_get_angles_as_dict(keypoints_coords, limbs):
    angle_information = {}
    
    for limb_1_name, limb_2_name in adjacent_limbs:
        limb_1 = limbs[limb_1_name]
        limb_2 = limbs[limb_2_name]
        
        common_point = None
        for point in limb_1:
            if any(np.array_equal(point, p) for p in limb_2):
                common_point = point
                break
        
        if common_point is None:
            continue
        
        vector_A = np.array(limb_1[1]) - np.array(limb_1[0]) if np.array_equal(limb_1[1], common_point) else np.array(limb_1[0]) - np.array(limb_1[1])
        vector_B = np.array(limb_2[1]) - np.array(limb_2[0]) if np.array_equal(limb_2[1], common_point) else np.array(limb_2[0]) - np.array(limb_2[1])
        
        angle = calculate_angle(vector_A, vector_B)
        key = f"{limb_1_name} - {KEYPOINT_LABELS[np.where((result_keypoint == common_point).all(axis=1))[0][0]]} - {limb_2_name}"
        value = f"{angle:.1f}°"
        
        angle_information[key] = value
        
    return angle_information

def find_closest_person_distance(prev_coordinates, current_coordinate):
    """
    Returns the index of the person from the previous frame whose coordinates 
    are closest to the current_coordinate and the corresponding distance.
    """
    distances = [(idx, np.linalg.norm(np.array(prev_coord) - np.array(current_coordinate)))
                 for idx, prev_coord in enumerate(prev_coordinates) if prev_coord is not None]
    if distances:
        min_distance_item = min(distances, key=lambda x: x[1])
        return min_distance_item
    return None, float('inf')

adjacent_limbs = []

selected_limbs = [
        "right_shoulder&right_elbow",
        "right_elbow&right_wrist",
        "left_shoulder&left_elbow",
        "left_elbow&left_wrist",
        "right_hip&right_knee",
        "right_knee&right_ankle",
        "left_hip&left_knee",
        "left_knee&left_ankle",
        "right_shoulder&left_shoulder",
        "right_hip&left_hip",
        "right_shoulder&right_hip",
        "left_shoulder&left_hip"
    ]

# Generating all possible combinations
for i in range(len(selected_limbs)):
    for j in range(i + 1, len(selected_limbs)):
        limb1 = limbs[selected_limbs[i]]
        limb2 = limbs[selected_limbs[j]]
        
        # Convert keypoints to tuples for comparison
        limb1_tuples = [tuple(point) for point in limb1]
        limb2_tuples = [tuple(point) for point in limb2]

        # Checking if any keypoint in the second limb matches with the keypoints in the first limb
        if any(kp in limb1_tuples for kp in limb2_tuples):
            adjacent_limbs.append((selected_limbs[i], selected_limbs[j]))

KEYPOINT_LABELS = [
    "NOSE", "LEFT_EYE", "RIGHT_EYE", "LEFT_EAR", "RIGHT_EAR", "LEFT_SHOULDER",
    "RIGHT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW", "LEFT_WRIST", "RIGHT_WRIST",
    "LEFT_HIP", "RIGHT_HIP", "LEFT_KNEE", "RIGHT_KNEE", "LEFT_ANKLE", "RIGHT_ANKLE"
]

angle_keys = [
    'right_shoulder&right_elbow - RIGHT_ELBOW - right_elbow&right_wrist',
    'right_shoulder&right_elbow - RIGHT_SHOULDER - right_shoulder&left_shoulder',
    'right_shoulder&right_elbow - RIGHT_SHOULDER - right_shoulder&right_hip',
    'left_shoulder&left_elbow - LEFT_ELBOW - left_elbow&left_wrist',
    'left_shoulder&left_elbow - LEFT_SHOULDER - right_shoulder&left_shoulder',
    'left_shoulder&left_elbow - LEFT_SHOULDER - left_shoulder&left_hip',
    'right_hip&right_knee - RIGHT_KNEE - right_knee&right_ankle',
    'right_hip&right_knee - RIGHT_HIP - right_hip&left_hip',
    'right_hip&right_knee - RIGHT_HIP - right_shoulder&right_hip',
    'left_hip&left_knee - LEFT_KNEE - left_knee&left_ankle',
    'left_hip&left_knee - LEFT_HIP - right_hip&left_hip',
    'left_hip&left_knee - LEFT_HIP - left_shoulder&left_hip',
    'right_shoulder&left_shoulder - RIGHT_SHOULDER - right_shoulder&right_hip',
    'right_shoulder&left_shoulder - LEFT_SHOULDER - left_shoulder&left_hip',
    'right_hip&left_hip - RIGHT_HIP - right_shoulder&right_hip',
    'right_hip&left_hip - LEFT_HIP - left_shoulder&left_hip'
]

def calculate_angle(A, B):
#     Calculate angle between two vectors A and B.
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return np.arccos(dot_product / (norm_A * norm_B)) * (180/np.pi)

prev_person_coordinates = [None, None]  # Assuming at most two employees, initialize with None
columns = ["Employee ID", "Box x", "Box y", "Box width", "Box height", "Width-to-height ratio"] + angle_keys + ["Fatigue or not"]

# columns = ["Employee ID", "Box x", "Box y", "Box width", "Box height", "Width-to-height ratio"] + [f"{keypoint} (x, y)" for keypoint in KEYPOINT_LABELS] + angle_keys + ["Fatigue or not"]

df = pd.DataFrame(columns=columns)
while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    
    # Process the frame with the YOLO model to get the keypoints and bounding boxes
    results = model.predict(frame, save=False, conf=0.5)
    
    # Iterate through results to get bounding boxes and keypoints data
    for idx, result in enumerate(results):
        keypoints_data = result.keypoints.xy.cpu().numpy() if hasattr(result, 'keypoints') else []
        boxes_data = result.boxes.xywh.cpu().numpy() if hasattr(result, 'boxes') else [] # Extract bounding boxes

        distances_and_indices = []

        # For each person, get the keypoints and determine which "employee" they match based on the nose's position
        for person_idx, result_keypoint in enumerate(keypoints_data):
            if len(result_keypoint) == 0:
                continue
            get_keypoint = GetKeypoint()
            nose_coordinate = tuple(result_keypoint[get_keypoint.NOSE])
            matched_person_idx, distance = find_closest_person_distance(prev_person_coordinates, nose_coordinate)
            distances_and_indices.append((distance, matched_person_idx, person_idx, result_keypoint))

        sorted_matches = sorted(distances_and_indices, key=lambda x: x[0])[:2]

        for distance, matched_person_idx, person_idx, result_keypoint in sorted_matches:
            if matched_person_idx is not None:
                prev_person_coordinates[matched_person_idx] = tuple(result_keypoint[get_keypoint.NOSE])

            body_keypoints = BodyKeypoints(result_keypoint)
            limbs = get_limbs_from_keypoints(body_keypoints)
            angle_info_dict = calculate_and_get_angles_as_dict(result_keypoint, limbs)

            # Construct the row data to be appended to the DataFrame
            row_data = {
                "Employee ID": person_idx + 1,
            }
    #         for label, keypoint in zip(KEYPOINT_LABELS, result_keypoint):
    #             row_data[f"{label} (x, y)"] = tuple(keypoint)
            for angle_key in angle_keys:
                angle_value = angle_info_dict.get(angle_key, None)
                if angle_value:
                    angle_value = ''.join([char for char in angle_value if char.isdigit() or char == '.'])
                row_data[angle_key] = angle_value

            # Include bounding box data for this person
            box = boxes_data[person_idx] if len(boxes_data) > person_idx else [None, None, None, None]
            # row_data["Bounding Box (x, y, w, h)"] = tuple(box)
            row_data["Box x"] = box[0] # The YOLO model frames the object in a box, and x refers to the x-coordinate of the diagonal intersection (geometric center) of the box.
            row_data["Box y"] = box[1]
            row_data["Box width"] = box[2]
            row_data["Box height"] = box[3]
            row_data["Width-to-height ratio"] = box[2]/box[3] # The width-to-height ratio of the box


            row_data['Fatigue or not'] = fatigue_or_not

            # Append the data to the DataFrame
            df = df.append(row_data, ignore_index=True)
            df = df.tail(1)
            df.drop(columns=['Fatigue or not'], errors='ignore', inplace=True)
            df = df.astype(float, errors='ignore')
            try:
                prediction = loaded_model.predict(df)
                if prediction[0] == 0:
                    prediction_result = 'Active'
                elif prediction[0] == 1:
                    prediction_result = 'Fatigue'
                else:
                    prediction_result = 'Unknown' 
            except Exception as e:
                print(f"An error occurred: {e}")
                break 
            if all(box):  # Check if bounding box data was obtained
                box_x, box_y, box_w, box_h = box
                top_left = (int(box_x - box_w/2), int(box_y - box_h/2))  # Calculate the coordinates of the top left corner of the bounding box
                bottom_right = (int(box_x + box_w/2), int(box_y + box_h/2))  # Calculate the coordinates of the bottom right corner of the bounding box

                # Draw the bounding box
                cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)  # Draw bounding box with green color lines

                # Draw the prediction text
                text_position = (top_left[0], top_left[1] - 10)  # Calculate the position of the text so it appears above the bounding box
                # Choose text color based on prediction result
                if prediction_result == 'Active':
                    text_color = (0, 255, 0)  # Green color
                else:
                    text_color = (0, 0, 255)  # Red color
                cv2.putText(frame, f"Status: {prediction_result}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)

        
    # Write the annotated frame to the output video
    output_video.write(frame)

# Release the video capture and writer
video_capture.release()
output_video.release()
cv2.destroyAllWindows()


0: 384x640 2 persons, 125.1ms
Speed: 7.1ms preprocess, 125.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 194.0ms
Speed: 6.9ms preprocess, 194.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  

0: 384x640 2 persons, 75.2ms
Speed: 1.6ms preprocess, 75.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 87.4ms
Speed: 1.8ms preprocess, 87.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

0: 384x640 2 persons, 84.8ms
Speed: 1.5ms preprocess, 84.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 79.4ms
Speed: 1.6ms preprocess, 79.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

0: 384x640 2 persons, 81.3ms
Speed: 2.0ms preprocess, 81.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 83.3ms
Speed: 1.4ms preprocess, 83.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

0: 384x640 2 persons, 77.5ms
Speed: 1.6ms preprocess, 77.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 85.9ms
Speed: 1.6ms preprocess, 85.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

...

0: 384x640 2 persons, 78.6ms
Speed: 1.5ms preprocess, 78.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 78.2ms
Speed: 1.5ms preprocess, 78.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

0: 384x640 2 persons, 78.4ms
Speed: 1.5ms preprocess, 78.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 80.7ms
Speed: 1.6ms preprocess, 80.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

0: 384x640 2 persons, 85.0ms
Speed: 1.7ms preprocess, 85.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 81.5ms
Speed: 1.6ms preprocess, 81.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

0: 384x640 2 persons, 79.2ms
Speed: 1.6ms preprocess, 79.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 84.2ms
Speed: 1.6ms preprocess, 84.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

0: 384x640 1 person, 78.0ms
Speed: 1.4ms preprocess, 78.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)

0: 384x640 2 persons, 80.9ms
Speed: 1.3ms preprocess, 80.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/n1/gzdnn7fx43324xp8l5rp6tth0000gn/T/ipykernel_74785/3182151652.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = d

In [7]:
import mysql.connector
from mysql.connector import errorcode

# Database connection configuration
URL = "localhost"
USER = "root"
PASSWORD = "Cc88888888@"

# Establishing a connection to the MySQL server
try:
    connection = mysql.connector.connect(
        host=URL,
        user=USER,
        password=PASSWORD
    )

    if connection.is_connected():
        print("Successfully connected to MySQL server")

        cursor = connection.cursor()

        # Creating the database if it does not exist
        cursor.execute("CREATE DATABASE IF NOT EXISTS tcs_db")
        print("Database tcs_db created successfully")

        # Switching to the database
        cursor.execute("USE tcs_db")

        # Dropping tables if they exist to avoid any conflicts
        cursor.execute("DROP TABLE IF EXISTS FATIGUE_HISTORY")
        cursor.execute("DROP TABLE IF EXISTS EMPLOYEE")
        print("Existing tables dropped successfully")

        # Creating the EMPLOYEE table
        create_employee_table_query = """
        CREATE TABLE EMPLOYEE (
            emp_id varchar(255) PRIMARY KEY,
            emp_name varchar(255),
            emp_position varchar(255)
        );
        """
        cursor.execute(create_employee_table_query)
        print("Table EMPLOYEE created successfully")

        # Creating the FATIGUE_HISTORY table
        create_fatigue_history_table_query = """
        CREATE TABLE FATIGUE_HISTORY (
            id INT AUTO_INCREMENT PRIMARY KEY,
            timestamp varchar(255),
            emp_id varchar(255),
            fatigue_status boolean,
            FOREIGN KEY (emp_id) REFERENCES EMPLOYEE(emp_id)
        );
        """
        cursor.execute(create_fatigue_history_table_query)
        print("Table FATIGUE_HISTORY created successfully")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



Successfully connected to MySQL server
Database tcs_db created successfully
Existing tables dropped successfully
Table EMPLOYEE created successfully
Table FATIGUE_HISTORY created successfully
MySQL connection is closed
